The purpose of this notebook is to assemble data from ABD estimation in each cluster together with average value of bioclimatic observables for each tree observation belonging to the cluster. The resulting dataset is used as training data for the ML model used to estimate ABD as a function of some bioclimatic observables. 

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from dataset_creation import add_feature_from_raster, add_feature_from_polygon_layer
from instance_grouping import aggregate_values_by_group

Load the tree dataset augmented with biogeographical realms and the ABD dataset.

In [ ]:
trees = pd.read_csv("/home/dibepa/git/global.agb.ml/data/training/tmp_preprocessed/tallo_clusters_agb.csv")

cluster_ABD = pd.read_csv("/home/dibepa/git/global.agb.ml/data/training/tmp_preprocessed/abd_cluster.csv") 

Add bioclimatic data on the tree dataset and save it.

In [ ]:
bioclim_data = [
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_isothermality.tif","iso"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_max_temp_warmest_month.tif","mtwm"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_mean_diurnal_range.tif","mdr"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_mean_temp_warmest_quarter.tif","mtwq"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_min_temp_coldest_month.tif","mtcm"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_precipitation_coldest_quarter.tif","pcq"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_precipitation_driest_month.tif","pdm"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_precipitation_driest_quarter.tif","pdq"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_precipitation_warmest_quarter.tif","pwaq"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_precipitation_wettest_quarter.tif","pweq"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_temperature_annual_range.tif","tar"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_annual_mean_temp.tif","yt"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_mean_temp_driest_quarter.tif","tdq"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_mean_temp_wettest_quarter.tif","twq"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_mean_temp_coldest_quarter.tif","tcq"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_annual_precipitation.tif","yp"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_precipitation_wettest_month.tif","pwm"),
    ("/home/dibepa/git/global.agb.ml/data/training/raw/bioclimatic_data/et0_v3_yr.tif","pet"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_temperature_seasonality.tif","ts"),
    ("/home/dibepa/Documents/worldclim_bioclim/HighResolution/wc2.1_30s_1970_2000_precipitation_seasonality.tif","ps"),
]

trees_geo = gpd.GeoDataFrame(
    trees, 
    geometry=gpd.points_from_xy(trees.longitude, trees.latitude)
)
trees.crs = "EPSG:4326"

for path,col in bioclim_data:
    trees_geo = add_feature_from_raster(trees_geo, col, path, "float")
    print(trees_geo)

trees_geo.to_csv("/home/dibepa/git/global.agb.ml/data/training/tmp_preprocessed/tallo_clusters_agb_bioclim.csv")

Calculate the mean, median and standard deviation of each bioclimatic observable within each tree cluster.

In [ ]:
# Drop all unnecessary information for the future. 
trees = pd.DataFrame(trees_geo.drop(
        ["reference_id",
         "Unnamed: 0",
         "geometry",
         "stem_diameter_cm",
         "height_m",
         "crown_radius_m",
         "tree_id",
         "division",
         "species",
         "genus",
         "family",
         "latitude",
         "longitude",
         "height_outlier",
         "crown_radius_outlier",
         "speciesCorrected",
         "genusCorrected",
         "meanWD",
         "height_local_m",
         "height_final_m",
         "agb_tropical",
         "agb_extra_tropical"]
         ,axis="columns")
    )

print(trees.columns)

# Create the dictionary with aggregation functions.
aggfunc_dict = {}

# Columns to get mean and standard deviation: 
agg_cols = ["yt","tdq","twq","tcq","yp","pwm","pet","iso","ts","ps","mtwm","mdr","mtwq","mtcm","pcq","pdm","pdq","pwaq","pweq","tar"]

# Columns to get first value (same value for all the group):
cluster_cols = ["bgr","bio","cluster"]

aggparams_list = [
    (cluster_cols, "first",False),
    (agg_cols,"mean",True),
    (agg_cols,"std",True)
]

trees_agg = aggregate_values_by_group(trees,"cluster",aggparams_list)

# Replace the standard deviation by the Coefficient of Variation.
std_cols = []
for col in agg_cols:
    cstd = col+"_std"
    cmean = col+"_mean"
    trees_agg[col+"_cv"] = np.abs(trees_agg[cstd]/trees_agg[cmean])
    std_cols.append(cstd)    

trees_agg = trees_agg.drop(std_cols,axis="columns")

print(trees_agg)


Include information on number of samples within each cluster.

In [ ]:
trees["n_samples"] = 1
cluster_samples = trees[["cluster","n_samples"]].groupby(["cluster"]).sum()

print(cluster_samples.columns)
print(trees_agg.columns)

trees_agg = trees_agg.set_index("cluster").join(cluster_samples).reset_index()

print(trees_agg)

trees_agg.to_csv("/home/dibepa/git/global.agb.ml/data/training/tmp_preprocessed/cluster_bioclimatic_summary.csv")

Now add information on ABD and tree density. 

In [ ]:
trees_agg = trees_agg.set_index("cluster").join(cluster_ABD.set_index("cluster"), how="inner").reset_index()

trees_agg.to_csv("/home/dibepa/git/global.agb.ml/data/training/tmp_preprocessed/cluster_bioclimatic_abd_summary.csv")

print(trees_agg)

Create the training dataset by removing CV columns, biome name and cluster ID.

In [ ]:
ABD_dataset = trees_agg[[
    'med_abd','bgr','yt_mean', 'tdq_mean', 'twq_mean', 'tcq_mean', 'yp_mean', 'pwm_mean', 'pet_mean', 'iso_mean','ts_mean', 'ps_mean','mtwm_mean', 'mdr_mean', 'mtwq_mean', 'mtcm_mean', 'pcq_mean','pdm_mean', 'pdq_mean', 'pwaq_mean', 'pweq_mean', 'tar_mean','n_samples'
]]

# Remove null ABDs caused by null densities probably caused by no-data in tree density or disalignment with tree density raster.

ABD_dataset = ABD_dataset.drop(ABD_dataset[ABD_dataset['med_abd']==0.0].index)

ABD_dataset = ABD_dataset.rename(
   columns = { 
        "med_abd" : "abd",
        "yt_mean" : "yt",
        "tdq_mean":"tdq",
        "twq_mean":"twq",
        "tcq_mean":"tcq",
        "yp_mean":"yp",
        "pwm_mean":"pwm",
        "pet_mean":"pet",
        "iso_mean":"iso",
        "ts_mean":"ts",
        "ps_mean":"ps",
        "mtwm_mean":"mtwm",
        "mdr_mean":"mdr",
        "mtwq_mean":"mtwq",
        "mtcm_mean":"mtcm",
        "pcq_mean":"pcq",
        "pdm_mean":"pdm",
        "pdq_mean":"pdq",
        "pwaq_mean":"pwaq",
        "pweq_mean":"pweq",
        "tar_mean":"tar"
    }
)

ABD_dataset["abd"] = ABD_dataset["abd"] * np.power(10.0,-5)

print(ABD_dataset)
trees_agg.to_csv("/home/dibepa/git/global.agb.ml/data/training/tmp_preprocessed/abd_bioclim_forested_regions.csv")

Add points for regions with null tree density with special focus on regions whose bioclimatic conditions are not comprised in the ABD dataset

In [ ]:
tree_density_file = "/home/dibepa/git/global.agb.ml/data/training/raw/tree_density/tree_density_biome_based_model_crowther_nature_2015_4326_float32.tiff" 

predictor_range_dir = "/home/dibepa/git/global.agb.ml/data/predict/predictor.range.onlybioclim/tmp"

print("Starting building the predictor out of range dataframe")

from natsort import natsorted
from pathlib import Path
import re
import rasterio

pfiles = natsorted( Path(predictor_range_dir).glob('predictors_in_range*.csv'), key=str )

df_predictor = pd.DataFrame()
for i,file in enumerate(pfiles):

    # Remove Antarctica
    window = re.findall('predictors_in_range(.*)', file.stem)[0] 
    lat = float(window.split("_")[-1])
    
    if lat>-60:

        df = pd.read_csv(file)

        df = df.drop( df[df.in_range > 98].index )

        if len(df.index)>0:

            df_predictor = pd.concat([df_predictor,df], axis = "rows")
    
    print("Processed {} out of {} files.".format(i+1,len(pfiles)))

df_predictor = df_predictor.reset_index(drop=True)
df_predictor = df_predictor.astype({'in_range':'uint16','x':'float32','y':'float32'})

print("Starting the creation of new 0 biomass locations")

samples = 0
points = pd.DataFrame() 
df_training = ABD_dataset.copy(deep=True)

tree_density = rasterio.open(tree_density_file)

while len(points.index)<len(df_training.index):
    
    s = df_predictor.sample()

    point = (s.x,s.y)
    id = s.index

    df_predictor.drop(id)

    td = [x for x in tree_density.sample([point])]

    if td[0] == 0 :
        p = pd.DataFrame({"x": s.x, "y": s.y})
        points = pd.concat([points,p], axis = "rows")
        samples +=1 
        print("Sampled {} points".format(samples))

points_gdf = gpd.GeoDataFrame(
            points,
            geometry = gpd.points_from_xy(x=points.x, y=points.y),
        )

points_gdf.crs = "EPSG:4326"

for path,col in bioclim_data:
    points_gdf = add_feature_from_raster(points_gdf, col, path, "float")
    print(points_gdf)

points_gdf = add_feature_from_polygon_layer(points_gdf,"REALM","/home/dibepa/git/global.agb.ml/data/training/raw/biome_data/Ecoregions2017.shp","bgr")
print(points_gdf)

zero_abd = pd.DataFrame(points_gdf.drop(columns=['geometry','x','y']))
zero_abd["abd"]=0.0
zero_abd['n_samples'] = np.median(ABD_dataset["n_samples"])

# Create new dataset

cols = zero_abd.columns

ABD = ABD_dataset[cols]

ABD = ABD.astype({'abd':'float32'})

ABD = pd.concat([ABD,zero_abd],axis="rows")
ABD = ABD.reset_index(drop=True)

print(ABD)

ABD.to_csv("/home/dibepa/git/global.agb.ml/data/training/detailed.allometries.model/ABD_training_dataset.csv")

In [6]:
tree_density_file = "/home/dibepa/git/global.agb.ml/data/training/raw/tree_density/tree_density_biome_based_model_crowther_nature_2015_4326_float32.tiff" 

predictor_range_dir = "/home/dibepa/git/global.agb.ml/data/predict/predictor.range.onlybioclim/tmp"

import rasterio
import numpy as np
import gc
from rasterio.windows import Window

tree_density = rasterio.open(tree_density_file)
td = tree_density.read(1)
rc = np.transpose(np.nonzero(td==0.0))
print(rc.shape)
xy = [tree_density.xy(row, col) for row, col in rc]
print(xy[0:5])




(81387149, 2)


KeyboardInterrupt: 

In [ ]:
points_gdf.to_csv("/home/dibepa/git/global.agb.ml/data/training/detailed.allometries.model/zero_biomass_points.csv")

ABD_dataset2 = ABD_dataset[ABD_dataset['abd']<5000]
print(np.max(ABD_dataset2['abd']))
print(np.min(ABD_dataset2['abd']))
print(np.mean(ABD_dataset2['abd']))
print(np.median(ABD_dataset2['abd']))
print(len(ABD_dataset2[ABD_dataset2['abd']<1].index))